# Scikit-decide performance

The aim of this notebook is to compare performance of scikit-decide and networkx on a given use case (shortest path in a graph with A* algorithm), try to explain differences and see if scikit-decide can run faster.  A* algorithm in network and scikit-decide are written in pure Python, and are very similar.

## Methodology

A `run_graph.py` script was provided, it runs each method several times with different start and end points, prints the path length to check that all methods return the same result, and prints processing time of each method.  The benchmarked algorithms were:

* `networkx`: calls `networkx.astar_path`
* `astar`: calls `skdecide.hub.solver.lazy_astar.LazyAstar`
* `stateful`: calls the same method, but `State` is defined as a class containing an `int`, whereas `State` is defined as an `int` in `astar`

It is important when profiling code to run only the portion of code of interest.  It is possible to comment out some parts, but it is then very difficult to reproduce the exact same commands. Command-line arguments have been added to `run_graph.py`:

    --repeat <int>         run searches for N different start and end points
    --algo networkx|astar  run only this algorithm (can be repeated)
    --output <filename>    write timings in a CSV file
    --seed <int>           set random generator seed

As shown above, `stateful` algorithm is not mentioned, it will be treated in a different manner.

Timings are stored in CSV files so that this notebook can display plots instantaneously.  We give arguments of `run_graph.py` script so that anyone can reproduce these results.  This script randomly selects a start and end node, and finds the shortest path between these nodes with the requested methods.

By default, each method is called 50 times with different start and end nodes, and we store in a CSV file the length of the solution (not used in this notebook, but may be useful) and the processing time of each method.  Profiling is run on 10 runs only, and on a single algorithm.  It is very important to understand that profiling has a very high overhead and may not be representative.  This gives hints to find hotspots, but benchmarking must always be performed without profiling.

Make sure that dependencies are installed. Bokeh is similar to matplotlib, but allows interactions (pan, zoom, etc).

In [ ]:
!pip install bokeh pandas numpy

Import bokeh.

In [ ]:
from bokeh.io import output_notebook
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show

Render plots directly in this notebook

In [ ]:
output_notebook()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def plot_timings(timings: pd.DataFrame, reference: str, colors) -> None:
    source = ColumnDataSource(timings)

    others = [k for k in timings.columns.values if k != reference and k in colors]
    ratio = ColumnDataSource(data=dict(
        index = range(len(timings)),
        **{ k: timings[k] / timings[reference] for k in others }
    ))

    p1 = figure()
    p1.xaxis.axis_label = 'Run number'
    p1.yaxis.axis_label = 'Time (s)'
    for algo in timings.columns.values:
        if algo not in colors:
            continue
        p1.line(x='index', y=algo, color=colors[algo], legend_label=algo, source=source)
        p1.line(x=range(len(timings)), y=np.mean(source.data[algo]), color=colors[algo], line_dash="dashed", legend_label=algo)
        p1.line(x=range(len(timings)), y=np.median(source.data[algo]), color=colors[algo], line_dash="dotted", legend_label=algo)
    p1.legend.location = "top_left"
    p1.legend.click_policy="hide"

    p2 = figure()
    p2.xaxis.axis_label = 'Run number'
    p2.yaxis.axis_label = f'Ratio time(algo)/time({reference})'
    for algo in others:
        p2.line(x='index', y=algo, color=colors[algo], legend_label=algo, source=ratio)
        p2.line(x=range(len(timings)), y=np.mean(ratio.data[algo]), color=colors[algo], line_dash="dashed", legend_label=algo)
        p2.line(x=range(len(timings)), y=np.median(ratio.data[algo]), color=colors[algo], line_dash="dotted", legend_label=algo)
    p2.legend.location = "top_left"
    p2.legend.click_policy="hide"

    show(row(p1, p2))

## Comparisons between scikit-decide and networkx

In [ ]:
# CSV file had been generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output ref.csv --seed 0
master = pd.read_csv("ref.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
master.head()

Define line colors.

In [ ]:
colors = {'networkx': 'green', 'astar': 'red'}

Left plot shows processing time of each method for different start and end nodes.  Dashed (resp. dotted) line is the mean (resp. median) on all runs. We see that scikit-decide is much slower than networkx, and the right plot shows how many times it is slower. Pikes are not meaningful, they always happen when processing time is very slow and thus timing is inaccurate, this is why median is also displayed.

In [ ]:
plot_timings(timings=master, reference="networkx", colors=colors)

On my computer, `astar` is about 2.8 times slower than `networkx`.  Even with the same random seed, timings may vary when running `run_graph.py` several times, it is a good idea plot timings of different runs and keep a representative one (on some run, your computer may get busy by doing some other stuff, for instance).  For illustration purpose, it may be interesting to gather timings of all `networkx` runs below into a single plot to display these differences.  But these differences do not matter much if one is careful to keep representative timings.
You may find different values when running `run_graph.py` on your computer, but normally all timings in this notebook should follow the same trend.

In order to improve performance, it is import to profile code and not rely on some intuition.  We run only `astar` method, on 10 iterations:

    python -m cProfile -o prof-ref.out run_graph.py --graph_file USA-road-d.NY.gr --repeat 10 --algo astar --seed 0

In [ ]:
import pstats

In [ ]:
p = pstats.Stats("prof-ref.out")
p.strip_dirs().sort_stats('cumtime').print_stats(20)

Here are some remarks about this output:
* First of all, we must be very cautious when reading these profiles.  They give hints to where to look at, but performance must always be checked afterwards without profiling.
* Output is not easy to read, this is not a call graph (graphic tools like snakeviz are useful to display a call graph)
* `solve` takes 22.6s, `extender` 15.6s and 7s are spent elsewhere
* `get_transition_value` is surprisingly heavy.  It creates a `Value` which inherits from `Generic`, and we see that most of the time is spent in `typing.py:868(__new__)`.  But one must be very cautious here: by looking at figures, one may think that this inheritance costs 4.1s out of 5.5s, but there are 1667334 calls to `get_transition_value` and 4008450 calls to `typing.py:868(__new__)`, so the real time spent in `__new__` is 4.129*1667334/4008450=1.7s when it is called from `get_transition_value`.  Still high, but not as high as it looked like first.


We will now display timings when `State` is a class (this was the `stateful` algorithm in original `run_graph.py`):

```
class GraphState:
    def __init__(self, id):
        self.id = id

    def __hash__(self):
        return self.id

    def __eq__(self, other):
        return self.id == other.id
```

In [ ]:
# CSV file had been generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output stateful.csv --seed 0
stateful = pd.read_csv("stateful.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=stateful, reference="networkx", colors=colors)

Now `astar` is 3.8 slower than `networkx` instead of 2.8.  This means that the overhead of using a class for `State` is not negligible.

In order to evaluate the cost of deriving from `Generic`, we make `GraphState` a dataclass similar to `Value`:
```
from dataclasses import dataclass
from typing import Generic
@dataclass
class GraphState(Generic[T]):
    id: T

    def __hash__(self):
        return self.id

    def __eq__(self, other):
        return self.id == other.id
```

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output dataclass-generic.csv --seed 0
dataclassgeneric = pd.read_csv("dataclass-generic.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=dataclassgeneric, reference="networkx", colors=colors)

`astar` now runs 4.35 times slower than `networkx`!

Here is a summary of our timings:

| State | comparison with networkx |
| --- | --- |
| int | x2.8 |
| normal class | x3.8 |
| dataclass with Generic | x4.35 |

In original profile we see that `get_next_state` is called twice `get_transition_value`, so if we find a way to change `Value` into a float, we may expect a slower gain, but that sounds interesting anyway.  `State` is set again to an `int`.

Instead of changing `Value` to a float, a simpler option in order to test this change is to introduce a method `_get_transition_cost` in `dynamics.py`.  This will break many things (we must choose between reward and cost, and C++ code has not been updated).  Again this change is applied only to have an idea of the speedup, so that developers can have insights to choose the best option.

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output value.csv --seed 0
value = pd.read_csv("value.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=value, reference="networkx", colors=colors)

There is a noticeable gain, slowdown is x2.4 instead of x2.8, but `heuristic` also creates `Value` instances.  We modify it in `lazy_astar.py` to return a float instead.

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output heuristic.csv --seed 0
heuristic = pd.read_csv("heuristic.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=heuristic, reference="networkx", colors=colors)

That is interesting, by replacing `Value` by a float, we were able to drop slowdown from x2.8 to almost x2.  We profile again after these changes:

    python -m cProfile -o prof-heuristic.out run_graph.py --graph_file USA-road-d.NY.gr --repeat 10 --algo astar --seed 0

In [ ]:
p = pstats.Stats("prof-heuristic.out")
p.strip_dirs().sort_stats('cumtime').print_stats(20)

We see that `get_applicable_actions` takes about 30% execution time of `extender`.  It is not clear from output above, but snakeviz shows that an important part of the time is spent in creating `ActionSpace` instances, because it inherits from `Generic`.  We remove this inheritance for all `*Space*` classes.

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output space.csv --seed 0
space = pd.read_csv("space.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=space, reference="networkx", colors=colors)

We modify `_get_applicable_actions` to call `ActionSpace` with a generator instead of a list.  It looks like result of getters are consumed once, thus a generator may be fine.

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output actions.csv --seed 0
actions = pd.read_csv("actions.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=actions, reference="networkx", colors=colors)

When looking at `extender` code, one see that two intermediate lists are created to return a third list

    def extender(node, label, explored):
       neigh = [(self._domain.get_next_state(node, a), a)
                 for a in self._domain.get_applicable_actions(node).get_elements()]
        neigh_not_explored = [(n, a) for n, a in neigh if n not in explored]
        cost_labels = [(n, self._domain.get_transition_cost(node, a, n), {'action': a})
                       for n, a in neigh_not_explored]
        return cost_labels

Its output is consumed in a `for` loop, so it can be rewritten as a generator without any intermediate list

    def extender(node, label, explored):
        for a in self._domain.get_applicable_actions(node).get_elements():
            n = self._domain.get_next_state(node, a)
            if n not in explored:
                yield (n, self._domain.get_transition_cost(node, a, n), {'action': a})

Replacing comprehension lists by generators is not always a good idea, readability may be more important than pure performance.

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output extender.csv --seed 0
extender = pd.read_csv("extender.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=extender, reference="networkx", colors=colors)

Ratio time goes from x1.75 down to x1.6, this change looks interesting.  Of course it will provide a lower speedup if applied on current `master` since the relative time spent in this portion of code is lower.

In [ ]:
p = pstats.Stats("prof-extender.out")
p.strip_dirs().sort_stats('cumtime').print_stats(20)

We see that `_get_applicable_actions_from` is still a hotspot.  In `extender` method, this an `ActionSpace` instance is created, but this is only a proxy. We replace

    (in ActionSpace)
    def _get_applicable_actions_from(self, memory: D.T_memory[D.T_state]) -> Space[D.T_event]:
        return ActionSpace(self.next_state_map[memory].keys())
    (in LazyAstar)
    def extender(node, label, explored):
       for a in self._domain.get_applicable_actions(node).getElements():
           ...

by

    (in ActionSpace)
    def _get_applicable_actions_from(self, memory: D.T_memory[D.T_state]) -> Iterable[D.T_event]:
        return self.next_state_map[memory].keys()
    (in LazyAstar)
    def extender(node, label, explored):
       for a in self._domain.get_applicable_actions(node):
           ...

To make this work, all `contains` methods must be renamed to `__contains__`.  BTW it is better to use Pythonic names.

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output contains.csv --seed 0
contains = pd.read_csv("contains.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=contains, reference="networkx", colors=colors)

## Discussion

Here is a summary of timings with different code changes:

| changes in code | comparison with networkx | relative speed-up |
| --- | --- | --- |
| reference | x2.8 | na |
| do not create Value instances | x2 | 1.4 |
| do not let `ActionSpace` inherit from `Generic` | x1.8 | 1.1 |
| pass a generator to `ActionSpace` | x1.75 | 1.1 |
| rewrite `extender` to not build lists | x1.6 | 1.1 |
| do not create ActionSpace instances | x1.5 | 1.1 |

Most of these changes tell the same story: creation of many instances of small objects may become a bottleneck.  This is not inherent to Python, for instance unboxing in Java induces the same performance problems.  In fact, this is a general problem with object oriented programming.  Is it more important to have clean code with very good encapsulation, or no encapsulation at all for best performances?
There is no definitive answer, it depends on your priority.  Current code is clean; if performance is not critical, it is reasonable to leave it as is.  If performance is an issue, then some design changes are necessary.

All methods defined in API (or at least the most used ones) should be examined to see if they really need to return a new instance.  If this instance is only a wrapper to an iterable, maybe it could return an `Iterable` instead?  If a class is really needed, another option is to pass a mutable argument instead of returning a new instance.  Of course this is ugly from an object-oriented programming perspective, this is why it must be balanced with expected performance gains.

For instance, if the `Value` class cannot be removed, we can replace

    def _get_transition_value(self, memory: D.T_memory[D.T_state],
                              event: D.T_event,
                              next_state: Optional[D.T_state] = None) -> Value[D.T_value]:
        return Value(cost=self.next_state_attributes[memory][event][self.attribute_weight])

by

    def _compute_transition_value(self, out: Value[D.T.value], memory: D.T_memory[D.T_state],
                                  event: D.T_event, next_state: Optional[D.T_state] = None) -> None:
        out.cost = self.next_state_attributes[memory][event][self.attribute_weight]

In order to test this solution, changes on `Value` are first reverted, then it uses `@property` decorator to handle mutation of cost and reward.  The `heuristic` function should also accept a mutable `Value` argument, but for simplicity reasons it returns a float instead.

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output compute.csv --seed 0
compute = pd.read_csv("compute.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=compute, reference="networkx", colors=colors)

This solution is a little bit slower than replacing `Value` by a float (x1.6 vs. x1.5), but it can take into account cost and reward.  For reference, here are timings when all other changes are applied but `Value` is kept as in `master`

In [ ]:
# CSV file generated by: python run_graph.py --graph_file USA-road-d.NY.gr --repeat 100 --output revert-value.csv --seed 0
revert = pd.read_csv("revert-value.csv", sep=" ", header=None, names=['length', 'networkx', 'astar'])
plot_timings(timings=revert, reference="networkx", colors=colors)